In [258]:
from utils.ETC import *
from matplotlib import pyplot as plt
from category_encoders import TargetEncoder
from sklearn.preprocessing import LabelEncoder
from IPython.core.interactiveshell import InteractiveShell

import re
import copy
import warnings
import pandas as pd

In [259]:
plt.rc('font', family='GULIM')
warnings.filterwarnings(action='ignore')
InteractiveShell.ast_node_interactivity = "all"

# Data Check

In [260]:
train = pd.read_csv('../Database/train.csv')
test = pd.read_csv('../Database/submission.csv', index_col=0)

train_target = pd.DataFrame(train['is_converted'])
test_target = pd.DataFrame(test['is_converted'])

delete_list = ['product_modelname', 'customer_country.1', 'is_converted']

continuous_list = ['com_reg_ver_win_rate', 'ver_win_ratio_per_bu', 'ver_win_rate_x', 'historical_existing_cnt',
                   'lead_desc_length']

weight_list = ['ver_cus', 'ver_pro', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 'enterprise',
               'lead_owner', 'response_corporate']

category_list = [item for item in train.columns if item not in continuous_list + weight_list + delete_list]

discrete_list = weight_list + category_list

clean_continuous_col = ['lead_desc_length']
unclean_continuous_col = [item for item in continuous_list if item not in clean_continuous_col]

clean_discrete_col = ['customer_idx', 'bant_submit', 'business_unit']
clean_discrete_col += weight_list
unclean_discrete_col = [item for item in discrete_list if item not in clean_discrete_col]

train_categorical = train[discrete_list]
test_categorical = test[discrete_list]

train.drop(columns=delete_list, inplace=True)
test.drop(columns=delete_list, inplace=True)
train.drop(columns=discrete_list, inplace=True)
test.drop(columns=discrete_list, inplace=True)

train = pd.concat([train_target, train, train_categorical])
test = pd.concat([test_target, test, test_categorical])

train = train.groupby(train.index).first()
test = test.groupby(test.index).first()

discrete_list.remove('customer_country')
discrete_list.remove('bant_submit')
discrete_list.append('country')

# General preprocessing

In [261]:
for col in weight_list + unclean_continuous_col:
    train[col] = train[col].fillna(0)
    test[col] = test[col].fillna(0)

for col in unclean_discrete_col:
    train[col] = train[col].fillna('none')
    test[col] = test[col].fillna('none')

for col in unclean_discrete_col:
    train[col] = train[col].str.replace('&', '/')
    test[col] = test[col].str.replace('&', '/')

    train[col] = train[col].apply(lambda x: re.sub('[^A-Za-z0-9가-힣/]', ' ', x))
    train[col] = train[col].apply(lambda x: re.sub(' +', '', x).strip())
    train[col] = train[col].str.lower()
    test[col] = test[col].apply(lambda x: re.sub('[^A-Za-z0-9가-힣/]', ' ', x))
    test[col] = test[col].apply(lambda x: re.sub(' +', '', x).strip())
    test[col] = test[col].str.lower()

In [262]:
train['country'] = train['customer_country'].str.split('/').str[-1]
test['country'] = test['customer_country'].str.split('/').str[-1]
train['country'] = train['country'].str.strip()
test['country'] = test['country'].str.strip()

country_mapping = {
    'uae': 'united arab emirates',
    'uaedubai': 'united arab emirates',
    'türkiye': 'trkiye',
    'us': 'united states',
    'usa': 'united states',
    'united states': 'united states',
    'uk': 'united kingdom',
    'brasil': 'brazil',
    'hanoi': 'vietnam',
    'southkorea': 'korea',
    'republicofkorea': 'korea',
    'skorea': 'korea',
    'czech': 'czech republic',
    'czechia': 'czech republic',
    'macau': 'china',
    'england': 'united kingdom',
    'scotland': 'united kingdom',
    'prc': 'china',
    'mainland china': 'china',
    'france': 'france',
    'french': 'france',
    'paris': 'france',
    'germany': 'germany',
    'deutschland': 'germany',
    'berlin': 'germany',
    'abudhabi': 'united arab emirates',
    'dubai': 'united arab emirates',
}

train['country'] = train['country'].replace(country_mapping)
test['country'] = test['country'].replace(country_mapping)

train.drop(columns='customer_country', inplace=True)
test.drop(columns='customer_country', inplace=True)

In [263]:
for col in discrete_list:
    train.loc[train[col] == 'others', col] = 'other'
    test.loc[test[col] == 'others', col] = 'other'
    train.loc[train[col] == 'etc', col] = 'other'
    test.loc[test[col] == 'etc', col] = 'other'
    train.loc[train[col] == '', col] = 'none'
    test.loc[test[col] == '', col] = 'none'

In [264]:
test.loc[test['customer_job'] == 'healthcare', 'customer_job'] = 'healthcareservices'
test.loc[test['customer_job'] == 'k12school', 'customer_job'] = 'teacher'
test.loc[test['product_category'] == 'notebook', 'product_category'] = 'laptop'
test.loc[test['product_category'] == 'ess', 'product_category'] = 'essentialseries'
test.loc[test['customer_position'] == 'pgtchemistry', 'customer_position'] = 'chemistryteacher'

train.loc[train['expected_timeline'] == '3months', 'expected_timeline'] = 'lessthan3months'
train.loc[train['expected_timeline'] == '45days', 'expected_timeline'] = 'lessthan3months'
train.loc[train['expected_timeline'] == '4/8months', 'expected_timeline'] = '3months6months'
train.loc[train['customer_position'] == 'enduser', 'customer_position'] = 'customer'

train.loc[train['customer_type'] == 'enduser', 'customer_type'] = 'endcustomer'
train.loc[train['customer_type'] == 'installer/contractor', 'customer_type'] = 'installer'

In [265]:
discrete_list.remove('lead_owner')

for col in discrete_list:
    frequency = train[col].value_counts()
    replace_train = frequency[frequency <= 10].index
    train[col] = train[col].apply(lambda x: x if x not in replace_train else 0)

    unique_train_values = train[col].unique()
    test[col] = test[col].apply(lambda x: x if x in unique_train_values else -1)

    # unique_test_values = test[col].unique()
    # train[col] = train[col].apply(lambda x: x if x in unique_test_values else -1)

for col in ['lead_owner']:
    frequency = train[col].value_counts()
    replace_train = frequency[frequency <= 5].index
    train[col] = train[col].apply(lambda x: x if x not in replace_train else 0)

    unique_train_values = train[col].unique()
    test[col] = test[col].apply(lambda x: x if x in unique_train_values else -1)

    # unique_test_values = test[col].unique()
    # train[col] = train[col].apply(lambda x: x if x in unique_test_values else -1)

discrete_list.append('lead_owner')

In [266]:
print('범주 갯수')
for i, name in enumerate(discrete_list):
    train_class = train[name].value_counts()
    test_class = test[name].value_counts()
    print(f'{name}: {len(set(train_class.index))} 개')
    print(f'{name}: {len(set(test_class.index))} 개')

    not_in_train = [item for item in test_class.index if item not in train_class.index]
    test_only_counts = test[name].isin(not_in_train).sum()
    test_only_ratio = test_only_counts / test[name].value_counts().sum()

    if not_in_train:
        print(f'{name}: Train에 없는 Test 범주갯수: {len(set(not_in_train))}개')
        print(f'{name}: Train에 없는 Test 범주갯수: {(set(not_in_train))}')

    else:
        print(f'{name}: 모든 Test 범주가 Train에 존재함.')
    print(f"Only Test 범주의 비율: {test_only_ratio * 100:.2f}%")
    print('')

범주 갯수
ver_cus: 2 개
ver_cus: 2 개
ver_cus: 모든 Test 범주가 Train에 존재함.
Only Test 범주의 비율: 0.00%

ver_pro: 2 개
ver_pro: 2 개
ver_pro: 모든 Test 범주가 Train에 존재함.
Only Test 범주의 비율: 0.00%

id_strategic_ver: 2 개
id_strategic_ver: 2 개
id_strategic_ver: 모든 Test 범주가 Train에 존재함.
Only Test 범주의 비율: 0.00%

it_strategic_ver: 2 개
it_strategic_ver: 2 개
it_strategic_ver: 모든 Test 범주가 Train에 존재함.
Only Test 범주의 비율: 0.00%

idit_strategic_ver: 2 개
idit_strategic_ver: 2 개
idit_strategic_ver: 모든 Test 범주가 Train에 존재함.
Only Test 범주의 비율: 0.00%

enterprise: 2 개
enterprise: 2 개
enterprise: 모든 Test 범주가 Train에 존재함.
Only Test 범주의 비율: 0.00%

response_corporate: 47 개
response_corporate: 41 개
response_corporate: Train에 없는 Test 범주갯수: 1개
response_corporate: Train에 없는 Test 범주갯수: {-1}
Only Test 범주의 비율: 0.02%

business_unit: 5 개
business_unit: 3 개
business_unit: 모든 Test 범주가 Train에 존재함.
Only Test 범주의 비율: 0.00%

customer_idx: 100 개
customer_idx: 42 개
customer_idx: Train에 없는 Test 범주갯수: 1개
customer_idx: Train에 없는 Test 범주갯수: {-1.0}
Only Tes

# Derivation Variables

In [82]:
train['ver_win_rate_x_category'] = train['ver_win_rate_x'].astype('object')
test['ver_win_rate_x_category'] = test['ver_win_rate_x'].astype('object')

train['historical_existing_cnt_category'] = train['historical_existing_cnt']
test['historical_existing_cnt_category'] = test['historical_existing_cnt']

frequency = train['historical_existing_cnt_category'].value_counts()
idx_to_replace = frequency[frequency <= 5].index
train['historical_existing_cnt_category'] = train['historical_existing_cnt_category'].replace(idx_to_replace, 'other')
train['historical_existing_cnt_category'] = train['historical_existing_cnt_category'].astype('object')

train_unique_values = {
    'historical_existing_cnt_category': set(
        train[train['historical_existing_cnt_category'] != 'other']['historical_existing_cnt_category'])
}

for col, unique_values in train_unique_values.items():
    test[col] = test[col].apply(lambda x: x if x in unique_values else 'other')

discrete_list.append('ver_win_rate_x_category')
discrete_list.append('historical_existing_cnt_category')

In [213]:
conversion_rate_and_count_by_lead_owner = train.groupby('lead_owner')['is_converted'].agg(
    ['mean', 'count']).sort_values(by='mean', ascending=False)
conversion_dict = conversion_rate_and_count_by_lead_owner['mean'].to_dict()
train['lead_owner_mean_value'] = train['lead_owner'].map(conversion_dict)
test['lead_owner_mean_value'] = test['lead_owner'].map(conversion_dict)

In [12]:
conditions = [
    ('new_column_90', (conversion_rate_and_count_by_lead_owner['mean'] >= 0.99) & (
            conversion_rate_and_count_by_lead_owner['count'] >= 5)),
    ('new_column_80', (conversion_rate_and_count_by_lead_owner['mean'] < 0.99) & (
            conversion_rate_and_count_by_lead_owner['mean'] >= 0.8) & (
             conversion_rate_and_count_by_lead_owner['count'] >= 5)),
    ('new_column_50', (conversion_rate_and_count_by_lead_owner['mean'] < 0.8) & (
            conversion_rate_and_count_by_lead_owner['mean'] >= 0.5) & (
             conversion_rate_and_count_by_lead_owner['count'] >= 5)),
    ('new_column_20', (conversion_rate_and_count_by_lead_owner['mean'] < 0.5) & (
            conversion_rate_and_count_by_lead_owner['mean'] >= 0.2) & (
             conversion_rate_and_count_by_lead_owner['count'] >= 5)),
    ('new_column_01', (conversion_rate_and_count_by_lead_owner['mean'] <= 0.01) & (
            conversion_rate_and_count_by_lead_owner['count'] >= 30))
]

for col_name, condition in conditions:
    qualified_lead_owners = conversion_rate_and_count_by_lead_owner[condition].index
    train[col_name] = 0
    test[col_name] = 0
    train.loc[train['lead_owner'].isin(qualified_lead_owners), col_name] = 1
    test.loc[train['lead_owner'].isin(qualified_lead_owners), col_name] = 1

In [75]:
train['response_corporate_high'] = 0
test['response_corporate_high'] = 0

train['response_corporate_low'] = 0
test['response_corporate_low'] = 0

train.loc[train['response_corporate'].isin(['LGEAF', 'LGEHK', 'LGERO']), 'response_corporate_high'] = 1
test.loc[test['response_corporate'].isin(['LGEAF', 'LGEHK', 'LGERO']), 'response_corporate_high'] = 1

train.loc[train['response_corporate'].isin(['LGEUK']), 'response_corporate_low'] = 1
test.loc[test['response_corporate'].isin(['LGEUK']), 'response_corporate_low'] = 1

In [76]:
train['business_subarea_high'] = 0
test['business_subarea_high'] = 0

train['business_subarea_low'] = 0
test['business_subarea_low'] = 0

train.loc[train['business_subarea'].isin(['hotel', 'cruise', 'hospital']), 'business_subarea_high'] = 1
test.loc[test['business_subarea'].isin(['hotel', 'cruise', 'hospital']), 'business_subarea_high'] = 1

train.loc[train['business_subarea'].isin(['logistics', 'bookstore']), 'business_subarea_low'] = 1
test.loc[test['business_subarea'].isin(['logistics', 'bookstore']), 'business_subarea_low'] = 1

In [77]:
train['country_high'] = 0
test['country_high'] = 0

train['country_low'] = 0
test['country_low'] = 0

train.loc[train['country'].isin(['nigeria', 'senegal', 'hongkong']), 'country_high'] = 1
test.loc[test['country'].isin(['nigeria', 'senegal', 'hongkong']), 'country_high'] = 1

train.loc[train['country'].isin(['italy', 'serbia', 'korea', 'afghanistan', 'trkiye']), 'country_low'] = 1
test.loc[test['country'].isin(['italy', 'serbia', 'korea', 'afghanistan', 'trkiye']), 'country_low'] = 1

In [78]:
train['customer_idx_high'] = 0
test['customer_idx_high'] = 0

train.loc[train['customer_idx'].isin([20943, 9857, 9324, 19252, 27735, 27737]), 'customer_idx_high'] = 1
test.loc[test['customer_idx'].isin([20943, 9857, 9324, 19252, 27735, 27737]), 'customer_idx_high'] = 1

In [79]:
train['lead_owner_high'] = 0
test['lead_owner_high'] = 0

train.loc[train['lead_owner'].isin(
    [368, 499, 608, 147, 152, 371, 448, 375, 438, 437, 372, 501, 839, 158, 585, 4, 603]), 'lead_owner_high'] = 1
test.loc[test['lead_owner'].isin(
    [368, 499, 608, 147, 152, 371, 448, 375, 438, 437, 372, 501, 839, 158, 585, 4, 603]), 'lead_owner_high'] = 1

In [157]:
# plot_top_categories_conversion_rate(train, 'country', 'is_converted', 0, 10)

In [81]:
train['customer_idx_row'] = 0
test['customer_idx_row'] = 0

train.loc[train['customer_idx'].isin([
    47466, 37680, 21321, 4936, 32240, 25309, 19804, 40491,
    42067, 37657, 31864, 33773, 37399, 40344, 7195, 18030,
    33334, 7810, 33350, 16590, 742, 9624
]), 'customer_idx_row'] = 1
test.loc[test['customer_idx'].isin([
    47466, 37680, 21321, 4936, 32240, 25309, 19804, 40491,
    42067, 37657, 31864, 33773, 37399, 40344, 7195, 18030,
    33334, 7810, 33350, 16590, 742, 9624
]), 'customer_idx_row'] = 1

In [267]:
# train['owner_avg_hist'] = train.groupby('lead_owner')['historical_existing_cnt'].transform('mean')
# test['owner_avg_hist'] = test.groupby('lead_owner')['historical_existing_cnt'].transform('mean')

train['desc_length_category'] = pd.cut(train['lead_desc_length'], bins=[0, 50, 150, 300, 450, np.inf],
                                       labels=[0, 1, 2, 3, 4])
test['desc_length_category'] = pd.cut(test['lead_desc_length'], bins=[0, 50, 150, 300, 450, np.inf],
                                      labels=[0, 1, 2, 3, 4])

train['desc_length_avg_win'] = train.groupby('desc_length_category')['ver_win_rate_x'].transform('mean')
test['desc_length_avg_win'] = test.groupby('desc_length_category')['ver_win_rate_x'].transform('mean')
# 
# train['business_area_avg_hist'] = train.groupby('business_area')['historical_existing_cnt'].transform('mean')
# test['business_area_avg_hist'] = test.groupby('business_area')['historical_existing_cnt'].transform('mean')
# 
# train['enterprise_avg_hist'] = train.groupby('enterprise')['historical_existing_cnt'].transform('mean')
# test['enterprise_avg_hist'] = test.groupby('enterprise')['historical_existing_cnt'].transform('mean')
# 
# train['corp_avg_hist'] = train.groupby('response_corporate')['historical_existing_cnt'].transform('mean')
# test['corp_avg_hist'] = test.groupby('response_corporate')['historical_existing_cnt'].transform('mean')

# train['country_avg_hist'] = train.groupby('country')['historical_existing_cnt'].transform('mean')
# test['country_avg_hist'] = test.groupby('country')['historical_existing_cnt'].transform('mean')

In [268]:
train.to_csv('../Database/test/train_before_encoding.csv')
test.to_csv('../Database/test/test_before_encoding.csv')

# Encoding Categories

In [269]:
train = pd.read_csv('../Database/test/train_before_encoding.csv', index_col=0)
test = pd.read_csv('../Database/test/test_before_encoding.csv', index_col=0)

continuous_list = ['com_reg_ver_win_rate', 'ver_win_ratio_per_bu', 'ver_win_rate_x', 'historical_existing_cnt',
                   'lead_desc_length']

weight_list = ['ver_cus', 'ver_pro', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver']

In [270]:
discrete_list = [item for item in discrete_list if item not in weight_list]
discrete_list.append('desc_length_category')

In [271]:
train_Copy = copy.deepcopy(train)
test_Copy = copy.deepcopy(test)

In [272]:
target_encoder1 = TargetEncoder(smoothing=10)
target_encoder2 = TargetEncoder(smoothing=10)
# discrete_list.remove('desc_length_category')

smoothing1_list = ['country', 'lead_owner']
smoothing2_list = [item for item in discrete_list if item not in smoothing1_list]

for col in smoothing1_list:
    train_Copy[col] = target_encoder1.fit_transform(train_Copy[col].astype('category'), train['is_converted'])
    test_Copy[col] = target_encoder1.transform(test_Copy[col].astype('category'))

for col in smoothing2_list:
    train_Copy[col] = target_encoder2.fit_transform(train_Copy[col].astype('category'), train['is_converted'])
    test_Copy[col] = target_encoder2.transform(test_Copy[col].astype('category'))

In [273]:
train_Copy['high_customer_idx'] = (train_Copy['customer_idx'] >= 0.98).astype(int)
train_Copy['high_lead_owner'] = (train_Copy['lead_owner'] >= 0.98).astype(int)

train_Copy['low_customer_idx'] = (train_Copy['customer_idx'] <= 0.0001).astype(int)
train_Copy['low_lead_owner'] = (train_Copy['lead_owner'] <= 0.0001).astype(int)

test_Copy['high_customer_idx'] = (test_Copy['customer_idx'] >= 0.98).astype(int)
test_Copy['high_lead_owner'] = (test_Copy['lead_owner'] >= 0.98).astype(int)

test_Copy['low_customer_idx'] = (test_Copy['customer_idx'] <= 0.0001).astype(int)
test_Copy['low_lead_owner'] = (test_Copy['lead_owner'] <= 0.0001).astype(int)

In [239]:
train.drop(columns=['idit_strategic_ver'], inplace=True)
test.drop(columns=['idit_strategic_ver'], inplace=True)

In [274]:
train_Copy.to_csv('../Database/test/train_k.csv')
test_Copy.to_csv('../Database/test/test_k.csv')